In [67]:
import pandas as pd
import pyspark 
import re
from datetime import datetime

In [70]:
requested_date=datetime(2025,3,6)
start_date=datetime(2025,1,1)

(requested_date-start_date).days

64

In [61]:
path='/Volumes/Macintosh HD/Users/carlossanchez/Documents/test_project/PIPELINES/energy-iso/Pr(Oficial)52_Viernes.xls'
df=pd.read_excel(path, sheet_name='Día 7')

In [62]:
df.head()

,VIERNES,Unnamed: 1,50,Unidad Marginando,Unnamed: 4,30,Generación Obligada,Unnamed: 7,0,Indisponible,...,Unnamed: 19,20,Prueba Parcial,Unnamed: 22,20.1,Prueba Total,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,2025-01-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14-11-2008,Predespacho Semana 52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pot.,NaN
2,Plantas,Precio,0-1,1-2,2-3,3-4,4-5,5-6,6-7,7-8,...,17-18,18-19,19-20,20-21,21-22,22-23,23-24,Total,MW,Costo ($)
3,MIRAFLORES G5,325.375,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,15.72,0
4,TROPITERMICA,284.231,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5.05,0


In [65]:
cols=df.columns.to_list()
df[cols[0]][0]

datetime.datetime(2025, 1, 3, 0, 0)

In [39]:
cleaned_columns = {
x: x.strip().translate(str.maketrans({
    "-": "_", 
    " ": "", 
    "(": "", 
    ")": "", 
    "[": "", 
    "]": "", 
    "$": "_USD"})).lower()
for x in df.columns.to_list()
}

In [41]:
df.rename(columns=cleaned_columns)

,plantas,precio,h0_1,h1_2,h2_3,h3_4,h4_5,h5_6,h6_7,h7_8,...,h20_21,h21_22,h22_23,h23_24,total,mw,costo_usd,file_id,file_name,epoch_public_date
0,MIRAFLORES G5,325.375,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.00,15.72,0.00,2855494,PrOficial9_Domingo_Sin_Restricciones.zip,1740872678340
1,TROPITERMICA,284.231,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.00,5.05,0.00,2855494,PrOficial9_Domingo_Sin_Restricciones.zip,1740872678340
2,J. BROWN G6,270.311,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.00,28.56,0.00,2855494,PrOficial9_Domingo_Sin_Restricciones.zip,1740872678340
3,TERMOCOLON G2,269.477,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.00,49.56,0.00,2855494,PrOficial9_Domingo_Sin_Restricciones.zip,1740872678340
4,TERMOCOLON G1,268.793,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.00,49.37,0.00,2855494,PrOficial9_Domingo_Sin_Restricciones.zip,1740872678340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,Exportación,NaN,63.000,119.900,121.300,121.100,120.800,129.700,127.300,178.000,...,197.300,195.500,177.500,142.100,3636.30,NaN,1251588.94,2855494,PrOficial9_Domingo_Sin_Restricciones.zip,1740872678340
161,Generación,NaN,1382.200,1395.900,1374.500,1348.000,1289.700,1237.600,1210.500,1280.400,...,1570.600,1570.600,1524.400,1430.700,33784.50,NaN,NaN,2855494,PrOficial9_Domingo_Sin_Restricciones.zip,1740872678340
162,CMS (B./),NaN,78.573,78.573,78.573,78.573,78.573,78.573,78.573,78.573,...,78.573,78.573,78.573,78.573,76.34,NaN,NaN,2855494,PrOficial9_Domingo_Sin_Restricciones.zip,1740872678340
163,R Disponible (MW),NaN,799.520,712.380,678.080,687.210,648.910,630.430,605.380,585.000,...,838.520,838.960,802.690,692.500,NaN,NaN,NaN,2855494,PrOficial9_Domingo_Sin_Restricciones.zip,1740872678340
